In [ ]:
import argparse
import ast
import multiprocessing as mp
import sys
from collections import defaultdict
from pathlib import Path

import pandas as pd
import pysam
from tqdm.auto import tqdm
from tracertools.seq import barcode_from_read, insertion_from_read, read_sam

In [ ]:
def call_alleles_10x(param):
    # Setup
    intID, bam, barcode_start, barcode_end, sites, min_reads, extract_barcode, out, lock = param
    if len(sites) > 0:
        end = max(sites.values())
    else:
        end = barcode_end
    # Get iterator
    bamfile = pysam.readFile(bam, "rb")
    if extract_barcode:
        total_reads = bamfile.count(contig=intID)
        read_iter = tqdm(bamfile.fetch(intID), total=total_reads, mininterval=60, desc="TS")
    else:
        read_iter = bamfile.fetch(intID)
    # Process reads
    umi_counts = defaultdict(int)
    for read in read_iter:
        if (read.mapping_quality < 30 or
            read.reference_start > barcode_start or
            read.reference_end < end or
            'N' in read.cigarstring or
            not read.has_tag('CB') or
            not read.has_tag('UB')):
            continue
        # Get integration
        if extract_barcode:
            intID = barcode_from_read(read.query_sequence, read.cigarstring, barcode_start, barcode_end, read.reference_start)
        else:
            intID = read.reference_name
        # Get allele
        alleles = []
        for name, pos in sites.items():
            allele = insertion_from_read(read.query_sequence, read.cigarstring, pos, read.reference_start)
            if name == "EMX1" and allele == "CTTGGG":
                allele = "None"
            alleles.append(allele)
        key = (read.get_tag('UB'),read.get_tag('CB'), intID, *alleles)
        umi_counts[key] += 1
    bamfile.close()
    # Correct and aggregate UMIs
    if len(umi_counts) > 0:
        site_names = list(sites.keys())
        allele_counts = pd.DataFrame(umi_counts.keys(), columns=["UMI","cellBC","intID"] + site_names)
        allele_counts["readCount"] = umi_counts.values()
        del umi_counts
        # correct UMIs
        allele_counts = allele_counts.groupby(["intID","cellBC","UMI"] + site_names).agg(
            {"readCount":"sum"}).sort_values("readCount", ascending=False).reset_index()
        agg_dict = {site: 'first' for site in site_names} 
        agg_dict["readCount"] = "sum"
        allele_counts = allele_counts.groupby(["intID","cellBC","UMI"]).agg(agg_dict).reset_index()
        # collapse UMIs
        allele_counts = allele_counts.groupby(["intID","cellBC"] + site_names).agg(
        {"UMI":"size","readCount":"sum"}).reset_index()
        # filter alleles
        allele_counts = allele_counts.query(f"readCount >= {min_reads}")
        with lock:
            allele_counts.to_csv(out, mode='a', header=False, index=False)

def alleles_from_bam(bam,
                    out,
                    barcode_start,
                    barcode_end,
                    site_positions,
                    min_reads=10,
                    extract_barcode=False):
     # Parse the arguments
    sites = ast.literal_eval(site_positions)
    bamfile = pysam.readFile(bam, "rb")
    intIDs = [ref for ref in bamfile.references if "intID" in ref]
    bamfile.close()
    # Make output file
    lock = mp.Manager().Lock()
    pd.DataFrame(columns=["intID", "cellBC"] + list(sites.keys()) + ["UMI", "readCount"]).to_csv(out, index=False)
    # Process
    if extract_barcode:
        call_alleles((intIDs[0],bamfile,barcode_start,barcode_end,sites,min_reads,extract_barcode,out,lock))
    # Process in parallel
    else:
        with mp.Pool(processes=8) as pool:
            _ = list(tqdm(pool.imap_unordered(call_alleles,
            [(intID,bamfile,barcode_start,barcode_end,sites,min_reads,extract_barcode,out,lock) for intID in intIDs]),
            total=len(intIDs),mininterval=60, desc="TS"))

def has_cb_tag(bamfile):
    i = 0
    for read in bamfile.fetch(until_eof=True):
        i += 1
        if i > 1000:
            break
        if read.has_tag("CB"):
            return True
    return False

In [13]:
bam = "/lab/solexa_weissman/PEtracing_shared/250717_dian_kinetics/bam/1_S1.sorted.bam"
out = "/lab/solexa_weissman/PEtracing_shared/250717_dian_kinetics/bam/alleles.csv"
sites = {'RNF2':332,'HEK3':380,'EMX1':448}
barcode_start = 270
barcode_end = 300
if len(sites) > 0:
    end = max(sites.values())
else:
    end = barcode_end

False

In [ ]:

bamfile = pysam.readFile(bam, "rb")
intIDs = [ref for ref in bamfile.references if "intID" in ref]
has_cb_tag(bamfile)
reads = []
for read in bamfile.fetch(until_eof=True):
    if (read.mapping_quality < 30 or
        read.reference_start > barcode_start or
        read.reference_end < end or
        'N' in read.cigarstring):
        continue
    alleles = []
    for name, pos in sites.items():
        allele = insertion_from_read(read.query_sequence, read.cigarstring, pos, read.reference_start)
        if name == "EMX1" and allele == "CTTGGG":
            allele = "None"
        alleles.append(allele)
    reads.append((read, alleles))

molecules = defaultdict(list)

In [109]:
bam

'/lab/solexa_weissman/PEtracing_shared/250717_dian_kinetics/bam/1_S1.sorted.bam'

In [130]:
from tqdm.auto import tqdm

def call_alleles_bulk(bam, barcode_start, barcode_end, sites, min_reads=10, extract_barcode=False):
    bamfile = pysam.AlignmentFile(bam, "rb")
    molecules = defaultdict(lambda: { **{ name: None for name in sites.keys() }, "intID": None })
     # Process reads
    total_reads = bamfile.count()
    bamfile.reset()
    for read in tqdm(bamfile.fetch(until_eof=True), total=total_reads, mininterval=20, desc="TS"):
        if (read.is_secondary or read.mapping_quality < 30):
            continue
        if read.reference_start < barcode_start and read.reference_end > barcode_start:
            molecules[read.query_name]["intID"] = read.reference_name
        for name, pos in sites.items():
            allele = insertion_from_alignment(
                    read.query_sequence,
                    read.cigarstring,
                    pos,
                    read.reference_start
                )
            molecules[read.query_name][name] = allele
    # Aggregate alleles
    alleles = pd.DataFrame.from_dict(molecules, orient='index')
    alleles = alleles.dropna(axis=0, how='any')
    if "EMX1" in alleles.columns:
        alleles["EMX1"] = alleles["EMX1"].replace("CTTGGG", "None")
    alleles["readCount"] = 1
    alleles = alleles.groupby(["intID"] + list(sites.keys())).agg({"readCount":"sum"}).reset_index()
    alleles = alleles.query(f"readCount >= {min_reads}")
    return alleles


In [124]:
test

,readCount


In [129]:
test

,RNF2,HEK3,EMX1,intID,readCount
M04449:1627:000000000-GRK9M:1:1101:11218:4751,None,None,None,intID21,1
M04449:1627:000000000-GRK9M:1:2103:8137:6188,None,None,None,intID21,1
M04449:1627:000000000-GRK9M:1:1104:14821:5130,None,None,None,intID23,1
M04449:1627:000000000-GRK9M:1:1102:14708:22348,None,None,None,intID26,1
M04449:1627:000000000-GRK9M:1:2104:4744:21078,None,None,None,intID28,1
...,...,...,...,...,...
M04449:1627:000000000-GRK9M:1:1102:12457:22671,None,None,None,intID2125,1
M04449:1627:000000000-GRK9M:1:2104:27455:8783,None,None,None,intID2125,1
M04449:1627:000000000-GRK9M:1:1102:7013:15012,None,None,None,intID2133,1
M04449:1627:000000000-GRK9M:1:2104:25119:23542,None,None,CTGGTC,intID2156,1


In [133]:
test.readCount.value_counts()

readCount
2       468
3       284
4       204
5       179
6       134
       ... 
328       1
313       1
173       1
72        1
6604      1
Name: count, Length: 344, dtype: int64

In [132]:
test

,intID,RNF2,HEK3,EMX1,readCount
7,intID1039,None,AATCG,ACAAT,2
8,intID1039,None,AATCG,CCCTA,2
10,intID1039,None,AATCG,None,86
13,intID1039,None,ATCAA,ACAAT,2
19,intID1039,None,ATCAA,None,107
...,...,...,...,...,...
4475,intID945,None,None,CTTGTG,3
4476,intID945,None,None,CTTTGG,6
4478,intID945,None,None,GGACA,144
4481,intID945,None,None,None,6604


In [131]:
test = call_alleles_bulk(bam, barcode_start, barcode_end, sites, min_reads=2, extract_barcode=False)

TS: 736352it [00:08, 90059.85it/s]            


In [101]:
alleles.EMX1.value_counts().head(10)

EMX1
None      320539
ACAAT       9218
CCCTA       8708
AGTAC       6528
GGACA       6313
CCTTT       4021
CCGAT       3006
ATCAA       2660
ATTCG        849
CTTTGG       191
Name: count, dtype: int64

In [103]:
alleles

,intID,RNF2,HEK3,EMX1,readCount
0,intID1003,None,None,ATTCG,1
1,intID1003,None,None,None,1
2,intID1010,None,None,None,1
3,intID1017,None,None,None,1
4,intID1019,None,None,None,1
...,...,...,...,...,...
4495,intID971,None,None,None,1
4496,intID980,None,None,CTTCGG,1
4497,intID983,None,None,None,1
4498,intID993,None,AATCG,None,1


In [99]:
alleles

,RNF2,HEK3,EMX1,intID
M04449:1627:000000000-GRK9M:1:1101:11218:4751,None,None,None,intID21
M04449:1627:000000000-GRK9M:1:2103:8137:6188,None,None,None,intID21
M04449:1627:000000000-GRK9M:1:1104:14821:5130,None,None,None,intID23
M04449:1627:000000000-GRK9M:1:1102:14708:22348,None,None,None,intID26
M04449:1627:000000000-GRK9M:1:2104:4744:21078,None,None,None,intID28
...,...,...,...,...
M04449:1627:000000000-GRK9M:1:1102:12457:22671,None,None,None,intID2125
M04449:1627:000000000-GRK9M:1:2104:27455:8783,None,None,None,intID2125
M04449:1627:000000000-GRK9M:1:1102:7013:15012,None,None,None,intID2133
M04449:1627:000000000-GRK9M:1:2104:25119:23542,None,None,CTGGTC,intID2156


In [97]:
alleles

,RNF2,HEK3,EMX1,intID
M04449:1627:000000000-GRK9M:1:1101:11218:4751,None,None,CTTGGG,intID21
M04449:1627:000000000-GRK9M:1:2103:8137:6188,None,None,CTTGGG,intID21
M04449:1627:000000000-GRK9M:1:1104:14821:5130,None,None,CTTGGG,intID23
M04449:1627:000000000-GRK9M:1:1102:14708:22348,None,None,CTTGGG,intID26
M04449:1627:000000000-GRK9M:1:2104:4744:21078,None,None,CTTGGG,intID28
...,...,...,...,...
M04449:1627:000000000-GRK9M:1:1102:12457:22671,None,None,CTTGGG,intID2125
M04449:1627:000000000-GRK9M:1:2104:27455:8783,None,None,CTTGGG,intID2125
M04449:1627:000000000-GRK9M:1:1102:7013:15012,None,None,CTTGGG,intID2133
M04449:1627:000000000-GRK9M:1:2104:25119:23542,None,None,CTGGTC,intID2156


In [ ]:
alleles["read"]

In [89]:
df.EMX1.isna().mean()

np.float64(0.009717288074780988)

In [90]:
df.intID.isna().mean()

np.float64(0.0006142956936784624)

In [91]:
df.intID

M04449:1627:000000000-GRK9M:1:1104:5049:23886        intID1
M04449:1627:000000000-GRK9M:1:1101:13312:25402         None
M04449:1627:000000000-GRK9M:1:1103:5451:5863           None
M04449:1627:000000000-GRK9M:1:2103:26401:12620       intID7
M04449:1627:000000000-GRK9M:1:2102:29184:18662      intID15
                                                    ...    
M04449:1627:000000000-GRK9M:1:2102:20579:12626    intID2159
M04449:1627:000000000-GRK9M:1:2103:29440:18060         None
M04449:1627:000000000-GRK9M:1:1102:25467:9445          None
M04449:1627:000000000-GRK9M:1:1102:4110:10252     intID2163
M04449:1627:000000000-GRK9M:1:2103:9856:10696     intID2168
Name: intID, Length: 367901, dtype: object

In [86]:
df

,RNF2,HEK3,EMX1,intID
M04449:1627:000000000-GRK9M:1:1104:5049:23886,None,None,<NA>,intID1
M04449:1627:000000000-GRK9M:1:1101:13312:25402,None,None,CTTGGG,intID1
M04449:1627:000000000-GRK9M:1:1103:5451:5863,None,None,CTTGGG,intID4
M04449:1627:000000000-GRK9M:1:2103:26401:12620,None,None,<NA>,intID7
M04449:1627:000000000-GRK9M:1:2102:29184:18662,None,None,<NA>,intID15
...,...,...,...,...
M04449:1627:000000000-GRK9M:1:2102:20579:12626,None,None,<NA>,intID2159
M04449:1627:000000000-GRK9M:1:2103:29440:18060,None,ATTTA,CTTGGG,intID2159
M04449:1627:000000000-GRK9M:1:1102:25467:9445,None,None,CTTGGG,intID2162
M04449:1627:000000000-GRK9M:1:1102:4110:10252,None,None,CTTGGG,intID2163


In [ ]:
bamfile = pysam.readFile(bam, "rb")
# optional check for cell-barcodes, etc.
has_cb_tag(bamfile)

# 1) Gather reads by query name
pairs = defaultdict(list)
for read in bamfile.fetch(until_eof=True):
    # skip secondary/supplementary reads
    if read.is_secondary or read.is_supplementary:
        continue
    # only keep properly paired
    if not (read.is_paired and read.is_proper_pair):
        continue
    # stash mate
    pairs[read.query_name].append(read)

reads_with_alleles = []

# 2) Process each completed pair
for qname, recs in pairs.items():
    # skip if not exactly two ends
    if len(recs) != 2:
        continue

    r1, r2 = sorted(recs, key=lambda r: r.is_read2)  # now r1.is_read1, r2.is_read2

    # apply your filters to both reads
    if  (r1.mapping_quality < 30 or
        r2.mapping_quality < 30 or
        r1.reference_start > barcode_start or
        r1.reference_end < r2.reference_start or
        r2.reference_end < end):
        continue

    # 3) Extract alleles from each read
    #    (here we just extract separately; you could also merge)
    alleles_pair = []
    for read in (r1, r2):
        alleles = []
        for name, pos in sites.items():
            allele = insertion_from_read(
                read.query_sequence,
                read.cigarstring,
                pos,
                read.reference_start
            )
            # special‐case rename
            if pd.isna(allele):
                allele = "None"
            if name == "EMX1" and allele == "CTTGGG":
                allele = "None"
            alleles.append(allele)
        alleles_pair.append(alleles)

    reads_with_alleles.append(((r1, r2), alleles_pair))

In [38]:
reads_with_alleles[13000]

((<AlignedSegment('M04449:1627:000000000-GRK9M:1:1104:23845:11886', flags=99=0x63, ref='intID257', zpos=262, mapq=255, cigar='138M', ...)>,
  <AlignedSegment('M04449:1627:000000000-GRK9M:1:1104:23845:11886', flags=147=0x93, ref='intID257', zpos=360, mapq=255, cigar='88M6I50M2I4M', ...)>),
 [['None', 'None', 'None'], ['None', 'None', 'None']])

In [61]:
df = []
for (r1, r2), alleles_pair in reads_with_alleles:
    df.append({
            "r1": r1.query_name,
            "r2": r2.query_name,
            "allele1": alleles_pair[0][0],
            "allele2": alleles_pair[0][1],
            "allele3": alleles_pair[1][2],
    })
df = pd.DataFrame(df)

In [62]:
len(df)

363262

In [56]:
df

""


In [57]:
df.allele2.value_counts().head(50)

AttributeError: 'DataFrame' object has no attribute 'allele2'

In [65]:
len(reads)

0

In [64]:
df.allele2.value_counts().head(50)

allele2
None       283002
ATTTA       16737
CTTTG       13755
GCGCC       11849
ATCAA       10776
AATCG        8473
CTCTC        7432
GATAG        5662
GCAAG        5111
ACTTA          93
CTTTTG         38
ACGCC          32
ACTAG          17
ACTTG          16
ATTTTTA        13
ACCAA          13
ACAAG          12
ATTAA          10
ACTCG           9
CCTTG           9
CTTTTTG         8
ATTTTA          8
ATTTG           7
GTTTA           6
ATCTC           6
ATCAG           6
CCCTC           6
ATTTAA          5
AATCT           5
GCTAG           4
ATCA            4
ATCGA           4
CTTTA           4
GTGCC           4
GAGCC           4
GCCCC           3
CATTG           3
ATCAAA          3
GCAGG           3
AATTA           3
GTCAA           3
AGTTA           3
ATTTAG          3
ATTTT           3
CTCTCA          2
AATGG           2
ATTT            2
ATCTA           2
CCAAG           2
CGCTC           2
Name: count, dtype: int64

In [29]:
len(pairs)

364062

In [32]:
pairs

defaultdict(list,
            {'M04449:1627:000000000-GRK9M:1:1101:23282:18933': [<AlignedSegment('M04449:1627:000000000-GRK9M:1:1101:23282:18933', flags=99=0x63, ref='intID250', zpos=262, mapq=255, cigar='136M', ...)>,
              <AlignedSegment('M04449:1627:000000000-GRK9M:1:1101:23282:18933', flags=147=0x93, ref='intID250', zpos=362, mapq=255, cigar='86M6I49M5I4M', ...)>],
             'M04449:1627:000000000-GRK9M:1:1101:5122:18946': [<AlignedSegment('M04449:1627:000000000-GRK9M:1:1101:5122:18946', flags=99=0x63, ref='intID250', zpos=262, mapq=255, cigar='118M5I12M', ...)>,
              <AlignedSegment('M04449:1627:000000000-GRK9M:1:1101:5122:18946', flags=147=0x93, ref='intID250', zpos=365, mapq=255, cigar='15M5I68M6I50M2I4M', ...)>],
             'M04449:1627:000000000-GRK9M:1:1101:21792:18953': [<AlignedSegment('M04449:1627:000000000-GRK9M:1:1101:21792:18953', flags=99=0x63, ref='intID250', zpos=262, mapq=255, cigar='117M5I15M', ...)>,
              <AlignedSegment('M04449:16

In [27]:
reads_with_alleles

[]

In [24]:
read

<AlignedSegment('M04449:1627:000000000-GRK9M:1:2104:19226:28485', flags=141=0x8d, ref=None, zpos=-1, mapq=0, cigar=None, ...)>

In [23]:
reads

[]

In [ ]:
 call_alleles_10x(param):
    # Setup
    intID, bam, barcode_start, barcode_end, sites, min_reads, extract_barcode, out, lock = param
    if len(sites) > 0:
        end = max(sites.values())
    else:
        end = barcode_end
    # Get iterator
    bamfile = pysam.readFile(bam, "rb")
    if extract_barcode:
        total_reads = bamfile.count(contig=intID)
        read_iter = tqdm(bamfile.fetch(intID), total=total_reads, mininterval=60, desc="TS")
    else:
        read_iter = bamfile.fetch(intID)
    # Process reads
    umi_counts = defaultdict(int)
    for read in read_iter:
        if (read.mapping_quality < 30 or
            read.reference_start > barcode_start or
            read.reference_end < end or
            'N' in read.cigarstring or
            not read.has_tag('CB') or
            not read.has_tag('UB')):
            continue

In [ ]:
read = read_sam(bam)

In [ ]:
read

,query_name,is_read1,is_read2,ref,query_begin,ref_begin,map_qual,CIGAR,seq
0,M04449:1627:000000000-GRK9M:1:1104:5049:23886,True,False,intID1,0,262,255,10M2I22M2D102M,GCTGTGCAGCAACTCGCCGGAGTGTAGAATTATGTGTCACTTAATT...
1,M04449:1627:000000000-GRK9M:1:1104:5049:23886,False,True,intID1,0,262,0,None,ATCTCTTAGCCGCAAATAGGAGAGCAGATCGCGATCTACCGGTAGA...
2,M04449:1627:000000000-GRK9M:1:1101:13312:25402,True,False,intID1,0,362,0,None,GCTGTGCAGCCTTTCGCCTCTGGGCCCAGTACCGACTCGGTTAATT...
3,M04449:1627:000000000-GRK9M:1:1101:13312:25402,False,True,intID1,0,362,255,86M6I46M6I6M,GGGCCCAGACTGAGCACGACTTGGCAGAGGAAAGGAAGGCCTGCTT...
4,M04449:1627:000000000-GRK9M:1:1103:5451:5863,True,False,intID4,0,360,0,None,GCTGTGCAGCATTGACCGATGTGGAGAGCCCTTGCCGTCTTAAATT...
...,...,...,...,...,...,...,...,...,...
736347,M04449:1627:000000000-GRK9M:1:2104:7826:26556,False,True,unmapped,0,-1,0,None,ATGTCCTTAGCCGCTAATAGGTGAGCAGTTAACGATCCACCGCTTG...
736348,M04449:1627:000000000-GRK9M:1:2104:15804:26586,True,False,unmapped,0,-1,0,None,GCTGTGCAGCCTTTTGGCATGGGACGTTAAGGTAACAGAATTAATT...
736349,M04449:1627:000000000-GRK9M:1:2104:15804:26586,False,True,unmapped,0,-1,0,None,GTCCCTTGGCCCCAATAAGGGGAGCAGTTAAGATCCAACCGGTTAA...
736350,M04449:1627:000000000-GRK9M:1:2104:19226:28485,True,False,unmapped,0,-1,0,None,GCTGTGCAGCTAAGCTTCTCTAAGTAGGTAGATTAGACAGTTAATT...
